# EDA: тексты запросов по сессиям

In [1]:
# тексты запросов, где только цифры, с цифрами, короткие, длинные запросы (отдельно по сессиям)
# - количество слов в запросе, бренды (отдельно по сессиям)
# - как часто полностью совпадает с названием категорий (по расс левенштейна/напрямую)

In [2]:
import json

with open("420_searches.json") as js:
    searches = json.load(js)
searchesLength = len(searches)

In [3]:
queryLengths = {}
queryWordQuantities = {}

for i in range(searchesLength):
    lwords = len(searches[i]["query"].split())
    if not (lwords in queryWordQuantities):
        queryWordQuantities[lwords] = 1
    else:
        queryWordQuantities[lwords] += 1
    l = len(searches[i]["query"])
    if not (l in queryLengths):
        queryLengths[l] = 1
    else:
        queryLengths[l] += 1
        
queryWordQuantities = {k: v for k, v in sorted(queryWordQuantities.items(), key = lambda item: item[0])}
queryLengths = {k: v for k, v in sorted(queryLengths.items(), key = lambda item: item[0])}

In [4]:
from math import log

sum = 0
print("Количество символов в запросе:")
for k, v in queryLengths.items():
    sum += k * v
    print("{0:3} : {1:4} : ".format(k, v), "*" * int(log(v, 1.2)))
print("\nВ среднем: {0}".format(sum // searchesLength))

Количество символов в запросе:
  1 :   45 :  ********************
  2 :  194 :  ****************************
  3 :  503 :  **********************************
  4 : 1038 :  **************************************
  5 : 1733 :  ****************************************
  6 : 2129 :  ******************************************
  7 : 2574 :  *******************************************
  8 : 2885 :  *******************************************
  9 : 3303 :  ********************************************
 10 : 3561 :  ********************************************
 11 : 4244 :  *********************************************
 12 : 4808 :  **********************************************
 13 : 5073 :  **********************************************
 14 : 5091 :  **********************************************
 15 : 4932 :  **********************************************
 16 : 4622 :  **********************************************
 17 : 4429 :  **********************************************
 18 : 4326 :  ***

In [5]:
print("Количество слов в запросе:")
for k, v in queryWordQuantities.items():
    print("{0:3} : {1:5} : ".format(k, v), "*" * int(log(v, 1.2)))

Количество слов в запросе:
  1 : 11518 :  ***************************************************
  2 : 31182 :  ********************************************************
  3 : 25515 :  *******************************************************
  4 : 19316 :  ******************************************************
  5 :  9420 :  **************************************************
  6 :  4518 :  **********************************************
  7 :  2222 :  ******************************************
  8 :  1412 :  ***************************************
  9 :   883 :  *************************************
 10 :   595 :  ***********************************
 11 :   422 :  *********************************
 12 :   314 :  *******************************
 13 :   238 :  ******************************
 14 :   162 :  ***************************
 15 :   144 :  ***************************
 16 :   100 :  *************************
 17 :    84 :  ************************
 18 :    65 :  **********************
 

In [6]:
import re

brands = []
with open("brands.txt") as raw:
    for row in raw:
        brands.append(re.sub("[^а-яА-Яa-zA-Z0-9]", "", row).lower())
brands = sorted(list(set(brands)))

In [7]:
categories = []
with open("categories.txt") as raw:
    for row in raw:
        categories.append(re.sub("[^а-яА-Яa-zA-Z0-9]", "", row).lower())
categories = sorted(list(set(categories)))

In [8]:
#%pip install python-Levenshtein
from Levenshtein import distance as lev

In [9]:
errorful = []
numerical = 0
containingNumbers = []
containingBrands = []
onlyBrands = 0
equalToCategory = 0
similarToCategory = []
for i in range(searchesLength):
    q = searches[i]["query"]
    qStripped = re.sub("[^а-яА-Яa-zA-Z0-9]", "", q)
    if (not qStripped):
        errorful.append(q)
    elif (qStripped == re.sub("[^0-9]", "", qStripped)):
        numerical += 1
    else:
        if (qStripped != re.sub("[^а-яА-Яa-zA-Z]", "", qStripped)):
            containingNumbers.append(q)
        qLower = q.lower()
       
        containsBrand = False
        for brand in brands:
            if (qLower.find(brand) != -1):
                containsBrand = True
                if (qLower == brand):
                    onlyBrands += 1
                break
        if (containsBrand):
            containingBrands.append(q)
        
        mivLev = len(qLower)
        for category in categories:
            if (lev(qLower, category) < len(qLower) // 3):
                similarToCategory.append(q)
                if (qLower == category):
                    equalToCategory += 1
                    break

In [14]:
def percent(val):
    return val * 100 // searchesLength

print("Запросов, содержащих цифры -- {0} ({1}%). Из них состоящих из цифр -- {2} ({3}%)."
      .format(len(containingNumbers), percent(len(containingNumbers)), 
              numerical, percent(numerical)))
print("Запросов, содержащих названия бренда -- {0} ({1}%). Из них состоящих только из названия -- {2} ({3}%)."
      .format(len(containingBrands), percent(len(containingBrands)),
             onlyBrands, percent(onlyBrands)))
print("Запросов, совпадающих с названием категории -- {0} ({1}%)."
      .format(equalToCategory, percent(equalToCategory)))
print("Запросов, похожих на название категории (с малым расстоянием Левентшейна) -- {0} ({1}%)."
      .format(len(similarToCategory), percent(len(similarToCategory))))

Запросов, содержащих цифры -- 9127 (8%). Из них состоящих из цифр -- 52 (0%).
Запросов, содержащих названия бренда -- 36859 (34%). Из них состоящих только из названия -- 89 (0%).
Запросов, совпадающих с названием категории -- 233 (0%).
Запросов, похожих на название категории (с малым расстоянием Левентшейна) -- 2939 (2%).


In [11]:
print(errorful)

['去了野黑美', 'ᴄᴏᴜᴛᴜʀᴇ ᴄᴏʟᴏᴜʀ ᴄʟᴜᴛᴄʜ', '古兰', '滴葡萄柚缺', '化妆革命眼影盘', '双萃', '\U0001f970', '美容盒', '히알루로 닉 페이스 세럼', '𝖼𝗈𝗅𝗅𝖺𝗀𝖾𝗇', '手', '斯蒂拉', '资生堂', '罗曼德唇膏', '娇兰', '苍白的阴影', '眼影盘', '资生堂至关重要的完美', '永远弥补', '克里奥', 'بكرة تدليك الوجه', '乔治·阿玛尼', '娇韵诗双萃', '兰蔻奇迹', '理肤泉', 'ჰუკსამსოო', '睫毛膏兰蔻催眠', '双重磨损', '汤姆·福特阴影', '油性头发的洗发水', '巴宝莉', '资生堂基金会', '达因欣奶油', '偶像兰蔻', 'ё', '眼影', '资生堂眼霜', '伊夫·圣洛朗·伊特', '巴黎欧莱雅', '兰蔻', '迪奥', '肌肤之钥', '卸妆油', '牙膏', '面部防晒霜', '雅诗兰黛', '清洁面膜', '洗澡', '海蓝之谜', '倩碧霜', '毛巾', '胡达贝蒂', '倩碧甚至更好的临床黑斑校正剂和优化剂', '赫莲娜', '森赛', '娇韵诗', '香奈儿', '乔治·阿玛尼·西', '城市衰落阴影', '资生堂生物性能', '娇韵诗 双萃', '安利', '百优', '美容拳击', 'ｎ', '阿玛尼', '香奈儿女士香熏香水新奇可可小姐', '乔·马龙香水', '古奇有罪', '纪梵希', '倩碧', 'ĵ ø马龙香水', '欧莱雅洗发水', '指甲油']


In [15]:
print(similarToCategory[0:20])

['armani basi', 'elan карандаш для бровей', 'erbrorian', 'essencial', 'glamglpw', 'la roche porsay', 'misshq', 'peritfee', 'romanovamakeup ash', 'shesseido', 'snow beauty', 'solo', 'детский набор', 'концерт для искусственного загара', 'крем для теле', 'специальные предложения', 'constant delight 12в1', 'giorgio armani a', 'hydra beauty', 'john varvatos vi']
